Loading params (only first time)

In [16]:
# !gdown --id 1l62zbWUFTlp214SvK6eMwPQZAzcwoeBE

# %%bash
# mv RETFound_cfp_weights.pth params/

# Setting

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import timm

# torchとtimmのバージョン確認
print(torch.__version__)
print(timm.__version__)


%cd /Users/keimy/git/retfound/
import src.models_vit


2.0.1
0.9.12
/Users/keimy/git/retfound


# Inference

msgオブジェクトのとこ

strict=False　でも問題ないのかどうかエラーメッセージを確認

In [8]:

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])


def prepare_model(chkpt_dir, arch='vit_large_patch16'):
    # build model
    model = src.models_vit.__dict__[arch](
        img_size=224,
        num_classes=5,
        drop_path_rate=0,
        global_pool=True,
    )
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    # msg = model.load_state_dict(checkpoint['model'])

    return model

def run_one_image(img, model):

    x = torch.tensor(img)
    x = x.unsqueeze(dim=0)
    x = torch.einsum('nhwc->nchw', x)

    x = x.to(device, non_blocking=True)
    latent = model.forward_features(x.float())
    latent = torch.squeeze(latent)

    return latent



### Load a pre-trained model

In [ ]:
# download pre-trained RETFound
chkpt_dir = "params/RETFound_cfp_weights.pth"
model_ = prepare_model(chkpt_dir, 'vit_large_patch16')

device = torch.device('cuda')
model_.to(device)
print('Model loaded.')


### Load images and save latent feature

In [ ]:
from tqdm import tqdm

# npyファイルを読み込み
data = np.load('datasets/mini_fundus_images.npy')

id_list = []
laterality_list = []
feature_list = []
model_.eval()

for idx in tqdm(range(len(data))):
    id = data['ID'][idx]
    laterality = data['laterality'][idx]
    img = data['image'][idx]

    img = Image.fromarray(np.uint8(data['image'][idx]))
    img = img.resize((224, 224))
    img = np.array(img) / 255.

    assert img.shape == (224, 224, 3)

    # normalize by mean and sd
    # can use customised mean and sd for your data
    img = img - imagenet_mean
    img = img / imagenet_std

    latent_feature = run_one_image(img, model_)

    # リストにデータ追加
    id_list.append(id)
    laterality_list.append(laterality)
    feature_list.append(latent_feature.detach().cpu().numpy())


100%|██████████| 1000/1000 [00:54<00:00, 18.24it/s]


In [ ]:
feature_list[0]

array([-0.07898155, -0.01586455, -0.03622571, ..., -0.12980084,
        0.84016883, -0.16708198], dtype=float32)

In [ ]:
# latent_csv = pd.DataFrame({'Name':name_list, 'Latent_feature':feature_list})
# latent_csv.to_csv('Feature_latent.csv', index = False, encoding='utf8')

# 構造化配列として保存
first_image_shape = feature_list[0].shape
dtype = [('ID', 'U20'), ('laterality', 'U1'), ('feature', np.float32, first_image_shape)]
data = np.zeros(len(id_list), dtype=dtype)

data['ID'] = id_list
data['laterality'] = laterality_list
data['feature'] = feature_list

np.save('outputs/fundus_features.npy', data)

# データ確認のみ

In [17]:
from tqdm import tqdm

# npyファイルを読み込み
data = np.load('datasets/mini_fundus_images.npy')

id_list = []
laterality_list = []
feature_list = []

for idx in tqdm(range(len(data))):
    id = data['ID'][idx]
    laterality = data['laterality'][idx]
    img = data['image'][idx]

    img = Image.fromarray(np.uint8(data['image'][idx]))
    img = img.resize((224, 224))
    img = np.array(img) / 255.

    assert img.shape == (224, 224, 3)

    # normalize by mean and sd
    # can use customised mean and sd for your data
    img = img - imagenet_mean
    img = img / imagenet_std


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/mini_fundus_images.npy'